In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Input, Embedding, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
# 1. Product Recommendation System using Neural Collaborative Filtering
def build_ncf_model(num_users, num_products, embedding_size=50):
    """
    Neural Collaborative Filtering model for product recommendations
    """
    # User input
    user_input = Input(shape=(1,), name='user_input')
    user_embedding = Embedding(num_users, embedding_size, name='user_embedding')(user_input)
    user_vec = Flatten()(user_embedding)

    # Product input
    product_input = Input(shape=(1,), name='product_input')
    product_embedding = Embedding(num_products, embedding_size, name='product_embedding')(product_input)
    product_vec = Flatten()(product_embedding)

    # Merge layers
    concat = concatenate([user_vec, product_vec])
    dense1 = Dense(128, activation='relu')(concat)
    dropout1 = Dropout(0.2)(dense1)
    dense2 = Dense(64, activation='relu')(dropout1)
    dropout2 = Dropout(0.2)(dense2)
    output = Dense(1, activation='sigmoid')(dropout2)

    model = Model(inputs=[user_input, product_input], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# 2. Customer Churn Prediction using LSTM
def build_churn_prediction_model(sequence_length, num_features):
    """
    LSTM model for predicting customer churn based on purchase history
    """
    model = Sequential([
        Input(shape=(sequence_length, num_features)),  # Explicit Input layer
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 3. Product Image Classification using CNN
def build_product_classification_model(input_shape, num_classes):
    """
    CNN model for classifying product images by category
    """
    model = Sequential([
        Input(shape=input_shape), 
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# 4. Sales Forecasting using LSTM
def build_sales_forecast_model(sequence_length, num_features):
    """
    LSTM model for forecasting sales
    """
    model = Sequential([
        Input(shape=(sequence_length, num_features)), 
        LSTM(50, return_sequences=True),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dense(25, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Example data generation and preprocessing functions
def generate_sample_data():
    """
    Generate sample data for testing models
    """
    # User-Product Interaction Data
    n_users = 1000
    n_products = 500
    n_interactions = 5000
    
    interactions = {
        'user_id': np.random.randint(0, n_users, n_interactions),
        'product_id': np.random.randint(0, n_products, n_interactions),
        'rating': np.random.uniform(1, 5, n_interactions),
        'timestamp': pd.date_range(start='2023-01-01', periods=n_interactions, freq='h')
    }
    
    # Customer Purchase History Data
    n_customers = 1000
    sequence_length = 10
    
    purchase_history = {
        'customer_id': np.repeat(range(n_customers), sequence_length),
        'purchase_amount': np.random.normal(100, 30, n_customers * sequence_length),
        'days_since_last_purchase': np.random.randint(1, 100, n_customers * sequence_length),
        'products_bought': np.random.randint(1, 10, n_customers * sequence_length)
    }
    
    return pd.DataFrame(interactions), pd.DataFrame(purchase_history)

def preprocess_recommendation_data(interactions_df):
    """
    Preprocess data for recommendation system
    """
    # Encode users and products
    user_encoder = LabelEncoder()
    product_encoder = LabelEncoder()
    
    interactions_df['user_encoded'] = user_encoder.fit_transform(interactions_df['user_id']) # IS THIS RIGHT??
    interactions_df['product_encoded'] = product_encoder.fit_transform(interactions_df['product_id'])
    
    # Create training data
    X_user = interactions_df['user_encoded'].values
    X_product = interactions_df['product_encoded'].values
    y = (interactions_df['rating'] > 3).astype(int).values  # Convert to binary feedback
    
    return (X_user, X_product), y, user_encoder, product_encoder

def preprocess_churn_data(purchase_history_df):
    """
    Preprocess data for churn prediction
    """
    # Create sequences of customer behavior
    sequences = []
    labels = []
    
    for customer in purchase_history_df['customer_id'].unique():
        customer_data = purchase_history_df[purchase_history_df['customer_id'] == customer]
        
        if len(customer_data) >= 10:  # Ensure enough history
            sequence = customer_data[['purchase_amount', 'days_since_last_purchase', 'products_bought']].values
            sequences.append(sequence[:10])  # Take last 10 purchases
            
            # Define churn based on days since last purchase
            churn = 1 if customer_data['days_since_last_purchase'].iloc[-1] > 60 else 0
            labels.append(churn)
    
    return np.array(sequences), np.array(labels)

# Main execution
if __name__ == "__main__":
    # Generate sample data
    interactions_df, purchase_history_df = generate_sample_data()
    
    # 1. Recommendation System
    print("Training Recommendation System...")
    (X_user, X_product), y, user_encoder, product_encoder = preprocess_recommendation_data(interactions_df)
    
    num_users = len(np.unique(X_user))
    num_products = len(np.unique(X_product))
    
    ncf_model = build_ncf_model(num_users, num_products)
    ncf_model.fit(
        [X_user, X_product], 
        y,
        batch_size=64,
        epochs=5,
        validation_split=0.2
    )
    
    # 2. Churn Prediction
    print("\nTraining Churn Prediction Model...")
    X_sequences, y_churn = preprocess_churn_data(purchase_history_df)
    
    churn_model = build_churn_prediction_model(
        sequence_length=10,
        num_features=X_sequences.shape[2]
    )
    churn_model.fit(
        X_sequences,
        y_churn,
        batch_size=32,
        epochs=5,
        validation_split=0.2
    )

    print("\nModels trained successfully!")

    # Example business applications
    def get_product_recommendations(user_id, top_n=5):
        """
        Get product recommendations for a user
        """
        user_encoded = user_encoder.transform([user_id])
        all_products = np.arange(num_products)
        
        predictions = ncf_model.predict([
            np.repeat(user_encoded, num_products),
            all_products
        ])
        
        top_product_indices = np.argsort(predictions.flatten())[-top_n:]
        return product_encoder.inverse_transform(top_product_indices)

    def predict_customer_churn_risk(customer_sequence):
        """
        Predict churn risk for a customer
        """
        sequence = np.expand_dims(customer_sequence, axis=0)
        return churn_model.predict(sequence)[0][0]

    # Example usage
    print("\nExample Recommendations for user 1:")
    print(get_product_recommendations(0))
    
    print("\nExample Churn Risk for a customer sequence:")
    print(f"Churn Risk: {predict_customer_churn_risk(X_sequences[0]):.2%}")

Training Recommendation System...
Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4888 - loss: 0.6936 - val_accuracy: 0.4930 - val_loss: 0.6929
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6505 - loss: 0.6838 - val_accuracy: 0.5060 - val_loss: 0.7023
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6803 - loss: 0.6128 - val_accuracy: 0.5080 - val_loss: 0.7401
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7558 - loss: 0.5329 - val_accuracy: 0.5080 - val_loss: 0.8232
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7901 - loss: 0.4764 - val_accuracy: 0.5080 - val_loss: 0.9314

Training Churn Prediction Model...
Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.6235 - loss: 0.6424 - val_accuracy: 0.8400 - val_loss: 0.4991
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8306 - loss: 0.4904 - val_accuracy: 0.9200 - val_loss: 0.3265
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - a